Importing all important libraries

In [ ]:
import firebase_admin
from firebase_admin import credentials
import requests as r
from firebase_admin import db
import json
import pandas as pd

In [ ]:
databaseURL = "https://sansahack2-default-rtdb.firebaseio.com/"
x_ray_flux = "https://services.swpc.noaa.gov/json/goes/primary/xrays-1-day.json"
solar_flare_ace = "https://services.swpc.noaa.gov/json/ace/swepam/ace_swepam_1h.json"

In [ ]:
cred = credentials.Certificate("serviceConfig.json")
if not firebase_admin._apps:
  firebase_admin.initialize_app(cred, {
      "app"
      "databaseURL": databaseURL
  })

# Get a database reference to our posts
ref1 = db.reference('realtime/x_ray_flux')
ref2 = db.reference('realtime/solar_wind')
ref3 = db.reference('alerts/solar_flare')


In [ ]:
#request
x_ray_req = r.get(x_ray_flux)
solar_flare_req = r.get(solar_flare_ace)

#response
x_ray_resp = x_ray_req.json()

solar_flare_resp = solar_flare_req.json()

with open('x_ray.json', 'w') as json_file:
    json.dump(x_ray_resp, json_file)

with open('solar_wind.json', 'w') as json_file:
    json.dump(solar_flare_resp, json_file)

print("Done")

Done


In [ ]:
#determining if there's a solar flare

#find an average over past 24 hours
#find average over past 6 hours

#if current flare > average flare (24 hours)
#check if average over 6 hours>average over 24 hours, flares have definitely occured

df = pd.read_json("x_ray.json")

print(df)

                  time_tag  satellite  ...  electron_contaminaton      energy
0     2021-06-25T08:48:00Z         16  ...                  False  0.05-0.4nm
1     2021-06-25T08:48:00Z         16  ...                  False   0.1-0.8nm
2     2021-06-25T08:49:00Z         16  ...                  False  0.05-0.4nm
3     2021-06-25T08:49:00Z         16  ...                  False   0.1-0.8nm
4     2021-06-25T08:50:00Z         16  ...                  False  0.05-0.4nm
...                    ...        ...  ...                    ...         ...
2871  2021-06-26T08:43:00Z         16  ...                  False   0.1-0.8nm
2872  2021-06-26T08:44:00Z         16  ...                  False  0.05-0.4nm
2873  2021-06-26T08:44:00Z         16  ...                  False   0.1-0.8nm
2874  2021-06-26T08:45:00Z         16  ...                  False  0.05-0.4nm
2875  2021-06-26T08:45:00Z         16  ...                  False   0.1-0.8nm

[2876 rows x 7 columns]


In [ ]:
df2 = pd.read_json("solar_wind.json")
print(df2)

                time_tag  dsflag      dens      speed  temperature
0    2021-06-26T09:00:00       0  3.278969  373.84830    48923.262
1    2021-06-26T08:00:00       0  2.813893  380.70688    33511.863
2    2021-06-26T07:00:00       0  3.105202  378.35623    26306.730
3    2021-06-26T06:00:00       0  2.473381  376.91907    58685.516
4    2021-06-26T05:00:00       0  1.637354  392.22452    47881.285
..                   ...     ...       ...        ...          ...
703  2021-05-26T15:00:00       0  1.194139  349.71158    85873.914
704  2021-05-26T14:00:00       1  1.438353  345.99432    96774.156
705  2021-05-26T13:00:00       1  0.876702  334.82684    63033.637
706  2021-05-26T12:00:00       0  1.118588  342.30496    72164.172
707  2021-05-26T11:00:00       1  1.489230  302.70419    30185.801

[708 rows x 5 columns]


In [ ]:
#last item in dataset
print(df.last_valid_index())
last = df.tail(1)

2875


In [ ]:
print(df2.last_valid_index())
first = df2.head(1)

707


In [ ]:
last_t = last["time_tag"].to_string()
last_t = last_t.split("    ")
last_t = last_t[1]

In [ ]:
first = first["time_tag"].to_string()
first = first.split("    ")
first = first[1]

In [ ]:
import datetime
from datetime import datetime, timedelta

In [ ]:
format = "%Y-%m-%dT%H:%M:%SZ"
t = datetime.strptime(last_t, format)

yt = datetime.strftime(t - timedelta(1), format)

In [ ]:
print(last_t2)

2021-06-26T08:45:00Z


In [ ]:
print(yt)

2021-06-25T08:45:00Z


In [ ]:
last24 = df.loc[(df['time_tag'] >= yt) & (df['time_tag'] <= last_t)]

#slaps face when sees that this is data for last 24hours...

In [ ]:
#avg over 24 hours
print(last24["flux"].mean())

1.0712881075900111e-07


In [ ]:
#avg over last 6 hours
last6HoursDateTime = t - timedelta(hours = 6)
l6 = datetime.strftime(last6HoursDateTime, format)

In [ ]:
print(l6)
print(type(l6))

2021-06-26T02:45:00Z
<class 'str'>


In [ ]:
last6 = df.loc[(df['time_tag'] >= l6) & (df['time_tag'] <= last_t)]

In [ ]:
#avg over last 6 hours
print(last6["flux"].mean())

8.423902211119387e-08


In [ ]:
last6mean = last6["flux"].mean()
last24mean = last24["flux"].mean()

In [ ]:
bMax = 1e-6
cMax = 1e-5
mMax = 1e-4

def classify(mean):
  if (mean<bMax):
    return "B"
  elif (mean<cMax and mean>=bMax):
    return "C"
  elif (mean<mMax and mean>=cMax):
    return "M"
  elif (mean>=mMax):
    return "X"

In [ ]:
print(last6mean>last24mean)

False


In [ ]:
classify(last6mean)

'B'

In [ ]:
classify(last24mean)

'B'

In [ ]:
(last["flux"]>last24mean).bool() and classify(last6mean)>classify(last24mean)

False

In [ ]:
classify(float((last["flux"].to_string()).split("    ")[1]))

'B'

In [ ]:
timeStamp = datetime.strftime(datetime.now(), format)
print(timeStamp)

2021-06-26T08:57:55Z


In [ ]:
ref3.child(timeStamp).update({
    'solarFlareClass': classify(float((last["flux"].to_string()).split("    ")[1])),
    'last6HoursClass':  classify(last6mean),
    'last24HoursClass': classify(last24mean),
    'solarFlareDetected': (last["flux"]>last24mean).bool() and classify(last6mean)>classify(last24mean),
    'timeStamp': timeStamp,
})

In [ ]:
print(df2)

                time_tag  dsflag      dens      speed  temperature
0    2021-06-26T09:00:00       0  3.278969  373.84830    48923.262
1    2021-06-26T08:00:00       0  2.813893  380.70688    33511.863
2    2021-06-26T07:00:00       0  3.105202  378.35623    26306.730
3    2021-06-26T06:00:00       0  2.473381  376.91907    58685.516
4    2021-06-26T05:00:00       0  1.637354  392.22452    47881.285
..                   ...     ...       ...        ...          ...
703  2021-05-26T15:00:00       0  1.194139  349.71158    85873.914
704  2021-05-26T14:00:00       1  1.438353  345.99432    96774.156
705  2021-05-26T13:00:00       1  0.876702  334.82684    63033.637
706  2021-05-26T12:00:00       0  1.118588  342.30496    72164.172
707  2021-05-26T11:00:00       1  1.489230  302.70419    30185.801

[708 rows x 5 columns]


In [ ]:
print(first)

2021-06-26T09:00:00


In [ ]:
speedMean = df2["speed"].mean()
print(speedMean)

396.276509066092


In [ ]:
densityMean = df2["dens"].mean()
print(densityMean)

3.433597321221269


https://umbra.nascom.nasa.gov/spartan/the_solar_wind.html

average solar wind speed = 400
when there's solar wind of high class, wind speed between 600 and 1000

In [ ]:
#look at most recent alert from firebase db
#if class of solar flare is C7 --- X
#check if there's an increase/spike in solar wind speed + density
print(first)
print(last)

              time_tag  dsflag      dens     speed  temperature
0  2021-06-26T09:00:00       0  3.278969  373.8483    48923.262
                  time_tag  satellite  ...  electron_contaminaton     energy
2875  2021-06-26T08:45:00Z         16  ...                  False  0.1-0.8nm

[1 rows x 7 columns]


In [ ]:
(first["dens"]>densityMean).bool() and (first["dens"]>5).bool()

False

In [ ]:
(first["speed"]>speedMean).bool() and (first["speed"]>600).bool()

False